In [1]:
import os
import sys

current_directory = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(current_directory, '..')))

In [2]:
from deck_class.truco import Deck_of_Truco
from dataclasses import dataclass
import numpy as np

In [20]:
@dataclass
class Player:
    hand: list
    index: int
    
    
@dataclass
class Team:
    players: list
    index: int

In [71]:
class Truco_Game():
    
    def __init__(self, max_points=12):
        self._deck = Deck_of_Truco()        
        self._goal = max_points
        self._teams = self.define_teams_hands()
        
        
    def define_teams_hands(self):
        player_11 = Player(self._deck.draw_hand(), 0)
        player_12 = Player(self._deck.draw_hand(), 0)
        player_21 = Player(self._deck.draw_hand(), 1)
        player_22 = Player(self._deck.draw_hand(), 1)
        
        team_1 = Team([player_11, player_21], 0)
        team_2 = Team([player_21, player_22], 1)
        
        return [team_1, team_2]
    
        
    def start_state(self):
        team = 0
        player = 0
        first_team_player = (team, player)
        player_hand = self._teams[team].players[player].hand
        
        points_t1 = 0
        points_t2 = 0
        round_value = 1
        raise_call = 0
        
        return (first_team_player, player_hand, points_t1, points_t2, round_value, raise_call)
    
    
    def successor(self, state, action):
        player, hand, points_t1, points_t2, round_value, raise_call = state
        
        if action in hand:
            self._teams[player[0]].players[player[1]].hand = np.delete(hand, np.where(hand == action))
            
        elif action == 'run':
            if player[0] == 1:
                points_t2 += round_value
            else:
                points_t1 += round_value
        
        elif action == 'raise':
            raise_call = 1
            
            if round_value == 1:
                round_value = 3
            else:
                round_value += 3
        
        
        next_player = self.get_next_player(player)
        next_hand = self._teams[next_player[0]].players[next_player[1]].hand
        
        return (next_player, next_hand, points_t1, points_t2, round_value, raise_call)
    
    
    def get_next_player(self, indexes):
        player_index = indexes[1]
        team_index = indexes[0]
        
        if team_index == 1:
            new_player_index = (player_index + 1)%2
        else:
            new_player_index = player_index
        
        new_team_index = (team_index + 1)%2
        
        return (new_team_index, new_player_index)
    
    
    def possible_actions(self, state):
        player, hand, points_t1, points_t2, round_value, raise_call = state
         
        if raise_call:
            possible_actions = ['go', 'run']
        else:
            possible_actions = list(self._teams[player[0]].players[player[1]].hand)
            
        if round_value <= 9 and max(points_t1, points_t2) < 11:
            possible_actions.append('raise')
            
        return possible_actions
        
        
      
    def is_end(start, state):
        points_t1, points_t2 = state[2], state[3]
        return points_t1 >= 12 or points_t2 >= 12
        
 
        

In [72]:
game = Truco_Game()

In [73]:
s1 = game.start_state()

In [74]:
game.possible_actions(s1)

['D2', 'C4', 'H7', 'raise']

In [75]:
game.successor(s1, 'H7')

((1, 0), array(['SA', 'S3', 'HA'], dtype='<U2'), 0, 0, 1, 0)

In [65]:
game._teams[1].players[1].hand

array(['SA', 'H2'], dtype='<U2')

In [64]:
game._teams

[Team(players=[Player(hand=array(['C3', 'HA', 'HQ'], dtype='<U2'), index=1), Player(hand=array(['SJ', 'SQ', 'DK'], dtype='<U2'), index=-1)], index=1),
 Team(players=[Player(hand=array(['SJ', 'SQ', 'DK'], dtype='<U2'), index=-1), Player(hand=array(['SA', 'H2'], dtype='<U2'), index=-1)], index=-1)]